In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import os
import yaml
from orix.quaternion import Orientation, Rotation
import orix.plot
from orix.quaternion.symmetry import D6h
from orix.vector import Vector3d, Miller
from orix.crystal_map import Phase
from scipy.optimize import minimize
from diffpy.structure import Lattice, Structure
plt.rcParams["svg.fonttype"] = "none"
a = 0.6115
c = 1.141

In [3]:
class GonioPosition:
    def __init__(self, x_tilt, y_tilt, degrees=True) -> None:
        if degrees:
            self.x_tilt = np.radians(x_tilt)
            self.y_tilt = np.radians(y_tilt)
        else:
            self.x_tilt = x_tilt
            self.y_tilt = y_tilt


    def __repr__(self) -> str:
        return f"x_tilt: {np.rad2deg(self.x_tilt) :.2f}, y_tilt: {np.rad2deg(self.y_tilt) :.2f}"
            
class RotationGenerator:
    def __init__(
        self,
        new_gonjo_pos: GonioPosition,
        old_gonjo_pos: GonioPosition = GonioPosition(0, 0),
    ) -> None:
        self.new_gonjo_pos = new_gonjo_pos
        self.old_gonjo_pos = old_gonjo_pos

        self.alpha_0 = self.old_gonjo_pos.x_tilt
        self.beta_0 = self.old_gonjo_pos.y_tilt

        self.alpha = self.new_gonjo_pos.x_tilt - self.old_gonjo_pos.x_tilt
        self.beta = self.new_gonjo_pos.y_tilt - self.old_gonjo_pos.y_tilt

    def get_T1(self):
        a11 = 1
        a12 = 0
        a13 = 0
        a21 = 0
        a22 = np.cos(self.alpha)
        a23 = -np.sin(self.alpha)
        a31 = 0
        a32 = np.sin(self.alpha)
        a33 = np.cos(self.alpha)

        return np.array([[a11, a12, a13], 
                         [a21, a22, a23], 
                         [a31, a32, a33]])

    def get_T2(self):
        a11 = np.cos(self.beta)
        a12 = -np.sin(self.beta) * np.sin(self.alpha_0)
        a13 = np.sin(self.beta) * np.cos(self.alpha_0)
        a21 = np.sin(self.beta) * np.sin(self.alpha_0)
        a22 = np.cos(self.alpha_0) ** 2 + np.sin(self.alpha_0) ** 2 * np.cos(self.beta)
        a23 = np.sin(self.alpha_0) * np.cos(self.alpha_0) * (1 - np.cos(self.beta))
        a31 = -np.sin(self.beta) * np.cos(self.alpha_0)
        a32 = np.sin(self.alpha_0) * np.cos(self.alpha_0) * (1 - np.cos(self.beta))
        a33 = np.sin(self.alpha_0) ** 2 + np.cos(self.alpha_0) ** 2 * np.cos(self.beta)

        return np.array([[a11, a12, a13], 
                         [a21, a22, a23], 
                         [a31, a32, a33]])

    def get_full_rotation_matrix(self):
        return self.get_T1() @ self.get_T2()
    


def rotate_inplane(ori, angle ):
    rotatior = Rotation.from_axes_angles([0,0,-1], np.deg2rad(angle))

    return Orientation(rotatior * ori, symmetry=ori.symmetry)


def MTEX_2_Orix(mtex_ori):
    rot_axes_align = Rotation.from_axes_angles([0, 0, 1], np.deg2rad(30))
    
    return Orientation(rot_axes_align*(~mtex_ori), symmetry=mtex_ori.symmetry)


def transform_to_gonio(ori, new_gonio, old_gonio, tilt_axes_align_angle = 9.61):
    rotgen = RotationGenerator(new_gonio, old_gonio)
    rotator = Rotation.from_matrix(rotgen.get_full_rotation_matrix())
    
    ori = ~ori
    ori = rotate_inplane(ori, tilt_axes_align_angle)
    ori = Orientation(rotator * ori, symmetry = ori.symmetry)
    ori = rotate_inplane(ori, -tilt_axes_align_angle)
    ori = ~ori

    return ori


def get_ori_dict(fname):
    with open(fname) as f:
        ori_dict = yaml.full_load(f)
    return ori_dict

def remove_legend_stuff(fig, ax):
    handles, labels = fig.gca().get_legend_handles_labels()

    by_label = dict(zip(labels, handles))

    handles, labels = ax.get_legend_handles_labels()

    handle_list, label_list = [], []
    for handle, label in zip(handles, labels):
        if label in ["sa_circle", "sa_sector"]:
            continue
        if label not in label_list:
            
            handle_list.append(handle)
            label_list.append(label)
    fig.legend(handle_list, label_list, loc = "upper left", fontsize = 16)



def solve_Gonjo_position(current_orientation : Orientation,
                         current_gonjopos : GonioPosition, 
                         desired_zoneaxis : Miller):
    
    equivalent_zoneaxes = desired_zoneaxis.symmetrise()
    def smallest_angle_for_tilt(gonjo : np.ndarray, current_orientation : Orientation, return_ori  = False, angle = 9.61):
        
        new_gonio = GonioPosition(*gonjo)
        
        new_orientation = transform_to_gonio(current_orientation, new_gonio, current_gonjopos)
        
        angles = np.rad2deg((~new_orientation * equivalent_zoneaxes).angle_with(Miller(uvw=[0,0,1], phase = desired_zoneaxis.phase)))
        if return_ori:
            return np.min(angles), new_orientation
        else:
            return np.min(angles)

    res = minimize(smallest_angle_for_tilt, np.array([0,0]), args=current_orientation, bounds=((-30,30), (-30,30)), method = "Nelder-Mead")

    x_opt =res.x


    return GonioPosition(*x_opt), smallest_angle_for_tilt(x_opt, current_orientation, return_ori=True)
    
def Get_GonioPosition(grain, zoneaxis):
    oris_full_lamella = get_ori_dict("Full_lamella_oris_rotated180.txt")

    lattice = Lattice(a, a, c, 90,90,120)
    p = Phase("6/mmm", 191, structure=Structure(lattice=lattice))

    if len(zoneaxis) ==3:
        wanted_zoneaxis = Miller(uvw=zoneaxis,phase = p)
    elif len(zoneaxis) == 4:
        wanted_zoneaxis = Miller(UVTW=zoneaxis,phase = p)
    else:
        raise ValueError("Did not understand the wanted zoneaxis")
   
    current_gonjo = GonioPosition(3.2,2.4)
    current_ori_mtex = Orientation(oris_full_lamella[grain], symmetry=D6h)
    current_ori = MTEX_2_Orix(current_ori_mtex)

    gonjo, (angle, ori) = solve_Gonjo_position(current_ori, current_gonjo, wanted_zoneaxis)
    print(f"Solution for {grain}:")
    if angle > 0.1:
        print("No solution found in bounds")
    else:
        print(gonjo, f"Angle: {angle :.2f}")
    

def search_for_zoneaxis(zoneaxis):
    oris_full_lamella = get_ori_dict("Full_lamella_oris_rotated180.txt")

    lattice = Lattice(a, a, c, 90,90,120)
    p = Phase("6/mmm", 191, structure=Structure(lattice=lattice))

    if len(zoneaxis) ==3:
        wanted_zoneaxis = Miller(uvw=zoneaxis,phase = p)
    elif len(zoneaxis) == 4:
        wanted_zoneaxis = Miller(UVTW=zoneaxis,phase = p)
    else:
        raise ValueError("Did not understand the wanted zoneaxis")
   
    current_gonjo = GonioPosition(3.2,2.4)

    for grain in oris_full_lamella.keys():
        current_ori_mtex = Orientation(oris_full_lamella[grain], symmetry=D6h)
        current_ori = MTEX_2_Orix(current_ori_mtex)
        
        gonjo, (angle, ori) = solve_Gonjo_position(current_ori, current_gonjo, wanted_zoneaxis)
    
        if angle > 0.1:
            continue
        else:
            print(f"Solution found for {grain}:")
            print(gonjo, f"Angle: {angle :.2f}\n")

In [4]:
oris_x = get_ori_dict("tiltseries_mean_ori_rotated_180.txt")
oris_y = get_ori_dict("y_tiltseries_rotated180.txt")

grain_data = {"f":[],"m":[], "n":[], "o":[], "d":[]}
ref_gonjo = GonioPosition(0,0)

for key, value in oris_x.items():
    y_tilt = 1.8
    x_tilt = key[1:].replace("p",".")
    if "m" in x_tilt:
        x_tilt = x_tilt.replace("m","-")
    x_tilt = float(x_tilt)
    old_gonio = GonioPosition(x_tilt, y_tilt)
    
    ori_mtex = Orientation(value, symmetry=D6h)
    ori = MTEX_2_Orix(ori_mtex)        
    ori = transform_to_gonio(ori, ref_gonjo, old_gonio)
    
    grain = key[0]
    grain_data[grain].append(ori.data[0])

for key, value in oris_y.items():
    x_tilt = 2.5
    y_tilt = key[1:].replace("p",".")
    if "m" in y_tilt:
        y_tilt = y_tilt.replace("m","-")
    y_tilt = float(y_tilt)
    old_gonio = GonioPosition(x_tilt, y_tilt)

    ori_mtex = Orientation(value, symmetry=D6h)
    ori = MTEX_2_Orix(ori_mtex)        
    ori = transform_to_gonio(ori, ref_gonjo, old_gonio)
    
    grain = key[0]
    grain_data[grain].append(ori.data[0])


colors = ["tab:orange", "tab:blue", "tab:red", "tab:green", "tab:purple"]
fig = plt.figure(figsize=(12,3*2))

gs = GridSpec(2,3)

ax_x = fig.add_subplot(gs[0,2],projection = "ipf", symmetry =D6h, direction = Vector3d.xvector())
ax_y = fig.add_subplot(gs[1,2],projection = "ipf", symmetry =D6h, direction = Vector3d.yvector())
ax_z = fig.add_subplot(gs[:,:2],projection = "ipf", symmetry =D6h, direction = Vector3d.zvector())

ax_x.set_title("X", fontsize = 16)
ax_y.set_title("Y", fontsize = 16)
ax_z.set_title("Z", fontsize = 16)

for (key, value), color in zip(grain_data.items(), colors):

  
    grain_ori = Orientation(value, symmetry=D6h)
    
    grain_ori = grain_ori.reduce()  
    mean_ori  = Orientation(grain_ori.mean(),symmetry = ori.symmetry)
    
    angles = np.rad2deg(grain_ori.angle_with(mean_ori))



    mean_angle = angles.mean()

    size = 50

    grain_label = {"f": "f", "m": "g", "n": "j" , "o" :"i", "d" : "h"}

    ax_x.scatter(grain_ori, label = f"Grain ${grain_label[key]}$: {mean_angle :.2f}°", c = color, s = size)
    ax_y.scatter(grain_ori, c = color, s = size)
    ax_z.scatter(grain_ori, c = color, s = size)

    
remove_legend_stuff(fig, ax_x)
plt.tight_layout()

In [5]:
Get_GonioPosition("h", [0,0,1])

Solution for h:
x_tilt: 3.36, y_tilt: 2.46 Angle: 0.00


In [6]:
search_for_zoneaxis([2,-1,-1,0])

Solution found for 2:
x_tilt: -7.37, y_tilt: -10.17 Angle: 0.00

Solution found for a:
x_tilt: -3.48, y_tilt: 11.85 Angle: 0.00

Solution found for b:
x_tilt: 6.68, y_tilt: 4.03 Angle: 0.00

Solution found for 7:
x_tilt: -3.90, y_tilt: 25.05 Angle: 0.00

Solution found for e:
x_tilt: 17.19, y_tilt: 16.63 Angle: 0.00

Solution found for f:
x_tilt: 3.20, y_tilt: 2.46 Angle: 0.00

Solution found for g:
x_tilt: 25.83, y_tilt: -15.34 Angle: 0.00

Solution found for i:
x_tilt: -22.50, y_tilt: 10.18 Angle: 0.00

Solution found for l:
x_tilt: 4.98, y_tilt: -25.42 Angle: 0.00

Solution found for o:
x_tilt: -12.97, y_tilt: 27.79 Angle: 0.00

Solution found for p:
x_tilt: -14.50, y_tilt: 11.74 Angle: 0.00

Solution found for r:
x_tilt: -7.74, y_tilt: -17.93 Angle: 0.00

Solution found for s:
x_tilt: 6.04, y_tilt: -18.57 Angle: 0.00

Solution found for z:
x_tilt: -4.61, y_tilt: -1.67 Angle: 0.00

